# 1. Automated Zoho Email Retrieval Pipeline

## Overview
This notebook implements a secure and automated email data extraction pipeline from Zoho Mail API. It provides comprehensive error handling, rate limiting, and structured data output for downstream processing.

## Key Features
- **OAuth2 Authentication**: Secure API access using refresh tokens
- **Rate Limiting**: Respects API limits to prevent throttling
- **Error Handling**: Comprehensive error handling and recovery
- **Progress Tracking**: Real-time progress monitoring for large operations
- **Structured Output**: JSON format for easy downstream processing

## Security Considerations
- Credentials are loaded from external `credentials.json` file (never committed to git)
- Access tokens are refreshed automatically
- No sensitive data is logged or displayed

## Prerequisites
- Zoho Mail API credentials (`credentials.json`)
- Python packages: `requests`, `json`, `time`
- Valid Zoho account with API access

## Expected Output
- `zoho_emails.json`: Structured email data for analysis
- Comprehensive logging of the retrieval process
- Error reports for failed operations

## 1.1 Environment Setup and Dependencies

Import required libraries and set up the environment for Zoho API interaction.

In [ ]:
import requests
import json
import time
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("✅ Required libraries imported successfully")
print(f"Python version: {os.sys.version}")
print(f"Requests version: {requests.__version__}")

## 1.2 Credential Management

Securely load API credentials from external configuration file to prevent accidental exposure in version control.

In [ ]:
def load_credentials():
    """
    Load Zoho API credentials from external file
    
    Returns:
        dict: Credential dictionary with client_id, client_secret, refresh_token
        
    Expected credentials.json format:
    {
        "zoho": {
            "client_id": "your_client_id_here",
            "client_secret": "your_client_secret_here", 
            "refresh_token": "your_refresh_token_here"
        }
    }
    """
    credentials_file = Path('credentials.json')
    
    if not credentials_file.exists():
        print("❌ Error: credentials.json file not found!")
        print("Please create credentials.json with your Zoho API credentials.")
        print("See README.md for credential setup instructions.")
        raise FileNotFoundError("credentials.json file is required but not found")
    
    try:
        with open(credentials_file, 'r') as f:
            credentials = json.load(f)
        
        zoho_creds = credentials.get('zoho', {})
        required_keys = ['client_id', 'client_secret', 'refresh_token']
        
        for key in required_keys:
            if key not in zoho_creds:
                raise KeyError(f"Missing required credential: {key}")
        
        print("✅ Credentials loaded successfully")
        return zoho_creds
        
    except json.JSONDecodeError as e:
        print(f"❌ Error: Invalid JSON in credentials.json: {e}")
        raise
    except Exception as e:
        print(f"❌ Error loading credentials: {e}")
        raise

# Load credentials at startup
try:
    creds = load_credentials()
    client_id = creds['client_id']
    client_secret = creds['client_secret'] 
    refresh_token = creds['refresh_token']
    print("✅ Credentials loaded successfully")
except Exception as e:
    print("Failed to load credentials. Please check credentials.json file.")
    # Use placeholder values for demonstration (NEVER use real credentials here)
    client_id = 'YOUR_CLIENT_ID_HERE'
    client_secret = 'YOUR_CLIENT_SECRET_HERE'
    refresh_token = 'YOUR_REFRESH_TOKEN_HERE'

## 1.3 API Configuration

Set up API endpoints and configuration parameters for Zoho Mail API interaction.

In [ ]:
# Zoho API Configuration
token_url = 'https://accounts.zoho.com/oauth/v2/token'
api_base_url = 'https://mail.zoho.com/api/accounts'
output_file = '../data/zoho_emails.json'
limit = 100  # Email batch size per API call (max recommended by Zoho)

print("✅ API configuration set up")
print(f"Token URL: {token_url}")
print(f"API Base URL: {api_base_url}")
print(f"Output file: {output_file}")
print(f"Batch size: {limit} emails per request")

## 1.4 Authentication Functions

Implement OAuth2 token refresh and account ID retrieval functions for secure API access.

In [ ]:
def refresh_access_token():
    """
    Refresh OAuth2 access token using refresh token
    
    Returns:
        str: Fresh access token for API calls
        
    Raises:
        requests.HTTPError: If token refresh fails
        KeyError: If response doesn't contain expected token
    """
    print("🔄 Refreshing access token...")
    
    params = {
        'refresh_token': refresh_token,
        'client_id': client_id,
        'client_secret': client_secret,
        'grant_type': 'refresh_token'
    }
    
    try:
        response = requests.post(token_url, data=params, timeout=30)
        response.raise_for_status()
        
        token_data = response.json()
        if 'access_token' not in token_data:
            raise KeyError("No access_token in response")
            
        token = token_data['access_token']
        print("🔑 Access token refreshed successfully")
        return token
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Network error during token refresh: {e}")
        raise
    except KeyError as e:
        print(f"❌ Invalid token response format: {e}")
        raise
    except Exception as e:
        print(f"❌ Unexpected error during token refresh: {e}")
        raise

def get_account_id(headers):
    """
    Retrieve Zoho account ID for API calls
    
    Args:
        headers (dict): HTTP headers with authorization token
        
    Returns:
        str: Account ID for email API calls
        
    Raises:
        requests.HTTPError: If API call fails
        KeyError: If response format is unexpected
    """
    print("🔍 Retrieving account ID...")
    
    try:
        response = requests.get(api_base_url, headers=headers, timeout=30)
        response.raise_for_status()
        
        data = response.json()
        if 'data' not in data or not data['data']:
            raise KeyError("No account data in response")
            
        account_id = data['data'][0]['accountId']
        print(f"✅ Account ID retrieved: {account_id[:8]}...")  # Partial ID for privacy
        return account_id
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Network error retrieving account ID: {e}")
        raise
    except (KeyError, IndexError) as e:
        print(f"❌ Invalid account response format: {e}")
        raise
    except Exception as e:
        print(f"❌ Unexpected error retrieving account ID: {e}")
        raise

## 1.5 Email Retrieval Functions

Implement core email retrieval functionality with pagination and error handling.

In [ ]:
def get_emails(account_id, headers, folder_id='INBOX', start_index=0):
    """
    Retrieve emails from specified folder with pagination
    
    Args:
        account_id (str): Zoho account identifier
        headers (dict): HTTP headers with authorization
        folder_id (str): Email folder to retrieve from (default: INBOX)
        start_index (int): Starting index for pagination
        
    Returns:
        dict: Email data response from API
        
    Raises:
        requests.HTTPError: If API call fails
        KeyError: If response format is unexpected
    """
    emails_url = f"{api_base_url}/{account_id}/folders/{folder_id}/messages"
    
    params = {
        'start': start_index,
        'limit': limit,
        'include': 'attachments,headers'
    }
    
    try:
        response = requests.get(emails_url, headers=headers, params=params, timeout=30)
        response.raise_for_status()
        
        return response.json()
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Network error retrieving emails: {e}")
        raise
    except Exception as e:
        print(f"❌ Unexpected error retrieving emails: {e}")
        raise

def retrieve_all_emails(account_id, headers, max_emails=None):
    """
    Retrieve all emails with pagination and progress tracking
    
    Args:
        account_id (str): Zoho account identifier
        headers (dict): HTTP headers with authorization
        max_emails (int, optional): Maximum number of emails to retrieve
        
    Returns:
        list: All retrieved email data
    """
    all_emails = []
    start_index = 0
    total_retrieved = 0
    
    print(f"📧 Starting email retrieval (max: {max_emails or 'unlimited'})")
    
    while True:
        try:
            # Get batch of emails
            response = get_emails(account_id, headers, start_index=start_index)
            
            if 'data' not in response or not response['data']:
                print("📭 No more emails to retrieve")
                break
            
            emails = response['data']
            batch_size = len(emails)
            
            if batch_size == 0:
                print("📭 No emails in current batch")
                break
            
            # Add emails to collection
            all_emails.extend(emails)
            total_retrieved += batch_size
            
            print(f"📥 Retrieved batch: {batch_size} emails (Total: {total_retrieved})")
            
            # Check if we've reached the limit
            if max_emails and total_retrieved >= max_emails:
                print(f"✅ Reached maximum emails limit: {max_emails}")
                break
            
            # Move to next batch
            start_index += limit
            
            # Rate limiting: wait between requests
            time.sleep(1)
            
        except Exception as e:
            print(f"❌ Error retrieving emails at index {start_index}: {e}")
            break
    
    print(f"✅ Email retrieval completed: {total_retrieved} emails retrieved")
    return all_emails

## 1.6 Main Execution Pipeline

Execute the complete email retrieval pipeline with proper error handling and data persistence.

In [ ]:
def main_email_retrieval_pipeline():
    """
    Main pipeline for email retrieval from Zoho Mail API
    
    Returns:
        bool: True if successful, False otherwise
    """
    print("🚀 Starting Zoho Email Retrieval Pipeline")
    print("=" * 50)
    
    try:
        # Step 1: Get access token
        access_token = refresh_access_token()
        headers = {
            'Authorization': f'Zoho-oauthtoken {access_token}',
            'Content-Type': 'application/json'
        }
        
        # Step 2: Get account ID
        account_id = get_account_id(headers)
        
        # Step 3: Retrieve emails
        print("\n📧 Starting email retrieval...")
        emails = retrieve_all_emails(account_id, headers, max_emails=1000)  # Limit for demo
        
        if not emails:
            print("❌ No emails retrieved")
            return False
        
        # Step 4: Save to file
        print(f"\n💾 Saving {len(emails)} emails to {output_file}...")
        
        # Ensure data directory exists
        os.makedirs(os.path.dirname(output_file), exist_ok=True)
        
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump({
                'retrieval_timestamp': time.strftime('%Y-%m-%d %H:%M:%S'),
                'total_emails': len(emails),
                'emails': emails
            }, f, indent=2, ensure_ascii=False)
        
        print(f"✅ Successfully saved {len(emails)} emails to {output_file}")
        
        # Step 5: Summary
        print("\n📊 RETRIEVAL SUMMARY")
        print(f"Total emails retrieved: {len(emails):,}")
        print(f"Output file: {output_file}")
        print(f"File size: {os.path.getsize(output_file) / (1024*1024):.2f} MB")
        
        return True
        
    except Exception as e:
        print(f"❌ Pipeline failed: {e}")
        return False

# Execute the pipeline
if __name__ == "__main__":
    success = main_email_retrieval_pipeline()
    if success:
        print("\n🎉 Email retrieval pipeline completed successfully!")
    else:
        print("\n💥 Email retrieval pipeline failed!")
else:
    print("📋 Email retrieval pipeline ready for execution")

## 1.7 Data Validation and Quality Check

Validate the retrieved email data and check for any issues or anomalies.

In [ ]:
def validate_email_data(file_path):
    """
    Validate retrieved email data for completeness and quality
    
    Args:
        file_path (str): Path to the email data file
        
    Returns:
        dict: Validation results and statistics
    """
    print(f"🔍 Validating email data from {file_path}...")
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        emails = data.get('emails', [])
        
        if not emails:
            print("❌ No emails found in data file")
            return None
        
        # Basic statistics
        total_emails = len(emails)
        
        # Check for required fields
        required_fields = ['messageId', 'subject', 'fromAddress']
        missing_fields = {}
        
        for field in required_fields:
            missing_count = sum(1 for email in emails if field not in email or not email[field])
            if missing_count > 0:
                missing_fields[field] = missing_count
        
        # Date range analysis
        dates = []
        for email in emails:
            if 'sentTime' in email and email['sentTime']:
                try:
                    # Convert timestamp to readable date
                    timestamp = int(email['sentTime']) / 1000  # Convert from milliseconds
                    date = time.strftime('%Y-%m-%d', time.localtime(timestamp))
                    dates.append(date)
                except:
                    pass
        
        date_range = f"{min(dates)} to {max(dates)}" if dates else "No valid dates"
        
        # Validation results
        validation_results = {
            'total_emails': total_emails,
            'missing_fields': missing_fields,
            'date_range': date_range,
            'file_size_mb': os.path.getsize(file_path) / (1024*1024),
            'retrieval_timestamp': data.get('retrieval_timestamp', 'Unknown')
        }
        
        print("✅ Email data validation completed")
        print(f"Total emails: {total_emails:,}")
        print(f"Date range: {date_range}")
        print(f"File size: {validation_results['file_size_mb']:.2f} MB")
        
        if missing_fields:
            print(f"⚠️  Missing fields: {missing_fields}")
        else:
            print("✅ All required fields present")
        
        return validation_results
        
    except Exception as e:
        print(f"❌ Validation failed: {e}")
        return None

# Validate the retrieved data
if os.path.exists(output_file):
    validation_results = validate_email_data(output_file)
    if validation_results:
        print("\n📊 VALIDATION SUMMARY")
        for key, value in validation_results.items():
            print(f"{key}: {value}")
else:
    print(f"📁 Email data file not found: {output_file}")
    print("Run the main pipeline first to retrieve emails.")

## Summary

This notebook successfully implements a comprehensive email retrieval pipeline from Zoho Mail API with the following capabilities:

### ✅ **Completed Features**
- Secure OAuth2 authentication with automatic token refresh
- Paginated email retrieval with progress tracking
- Comprehensive error handling and recovery
- Rate limiting to respect API constraints
- Structured JSON output for downstream processing
- Data validation and quality checks

### 🔐 **Security Measures**
- External credential management
- No sensitive data in logs or output
- Secure token handling

### 📊 **Output**
- `zoho_emails.json`: Complete email dataset
- Validation reports and statistics
- Comprehensive logging of the process

### 🚀 **Next Steps**
The retrieved email data is now ready for processing in the next pipeline stage (Data Analysis and Cleaning).